# Calcul de la distance de Levenshtein

In [1]:
import numpy as np
def levenshtein(chaine1, chaine2):
    if len(chaine1) < len(chaine2):
        return levenshtein(chaine2, chaine1)

    if len(chaine2) == 0:
        return len(chaine2)

    
    chaine1 = np.array(tuple(chaine1))
    chaine2 = np.array(tuple(chaine2))


    ligne_preced = np.arange(chaine2.size + 1)
    for s in chaine1:
        #insertion:
        ligne_actuel = ligne_preced + 1

        # Substitution:
        ligne_actuel[1:] = np.minimum(
                ligne_actuel[1:],
                np.add(ligne_preced[:-1], chaine2 != s))

        # effacement:
        ligne_actuel[1:] = np.minimum(
                ligne_actuel[1:],
                ligne_actuel[0:-1] + 1)

        ligne_preced = ligne_actuel

    return ligne_preced[-1]
print('La distance de Levenshtein entre acceuil et accueil est:',levenshtein('acceuil', 'accueil'))
print('La distance de Levenshtein entre arborigène et aborigène est:',levenshtein('arborigène', 'aborigène'))
print('La distance de Levenshtein entre chiens et niche est:',levenshtein('chiens', 'niche'))
import time
start_time = time.time()
y = levenshtein('chiens', 'niche')

 
print ("Temps d'execution de Levenshtein(chiens,niche): ", time.time() - start_time,'s')

La distance de Levenshtein entre acceuil et accueil est: 2
La distance de Levenshtein entre arborigène et aborigène est: 1
La distance de Levenshtein entre chiens et niche est: 5
Temps d'execution de Levenshtein(chiens,niche):  0.0007665157318115234 s


# Le correcteur orthographique proposé

Pour reussir notre correcteur orthographique on va passer par trois étapes:  
    $*$On construit la table de hashcode correspondant au dictionnaire fournie.  
    $*$On lit une liste de mots corrects (le dictionnaire) pour chaque mot.  
    $*$La correction orthographique.
    

*Etape1*

Dans cette étape on va tout d'abord lire notre document:

In [2]:
fichier=open('/home/wyouness/Téléchargements/python/listeDesMotsFrancais','r')
s=fichier.read()

Après on va mettre les mots dans une liste:

In [3]:
listDesMots=s.split('\n')

Maintenant je vais choisir la fonction de hashage:
    

In [4]:
N = 1000000 
def hashage(mot):
    hash=sum(ord(s)for s in mot)%N
    return hash

on va représenterer les mots ayant le meme hashcode sous forme d'un dictionnaire:

In [5]:
listeDeHashage=set([hashage(elm)for elm in listDesMots])
import time
start_time = time.time()
lstMotsOntMemeHashe={hash:[mot  for mot in listDesMots if hashage(mot)==hash] for hash in listeDeHashage }
print ("Temps d'execution de cette operation est: ", time.time() - start_time,"s comme c'est long")

Temps d'execution de cette operation est:  1304.346748828888 s comme c'est long


*Etape 2*

Dans cette étape on va essayer de construire le dictionnaire des tri-grammes:  
    Tout d'abord on rajoute un caractère « $ » au début et à la fin des mots.

In [6]:
listeTrigram=[]
for mot in listDesMots:
    listeTrigram.append('$'+mot+'$')

Je vais construire la fonction de trigramme:

In [7]:
def tri(mot):

    s=set()
    for i in range(len(mot)-2):

        s.add(mot[i:i+3])
        
    return(s)

Après le dictionnaire des tri-grammes:

In [8]:
setTriGramme=set()
for tg in listeTrigram:
    if '$' not in tg:
        setTriGramme.update(tri(mot))
import time
start_time = time.time()
dicTrigramme={tg:{mot for mot in listDesMots if tg in mot}for tg in setTriGramme}
print ("Temps d'execution de cette operation est: ", (time.time() - start_time)/60,"minutes est ce que ça prend beaucoup du temps??")

Temps d'execution de cette operation est:  6.425380706787109e-06 minutes comme c'est looong!!!


Finalement passant à la fonction de correction orthographique :o
Mais j'ai un problème avec le coefficient de Jaccard! J'ai pas pu construire la fonction de Jaccard selon la page http://fr.wikipedia.org/wiki/Indice_et_distance_de_Jaccard j'ai essayé avec ça après je me bloque!




In [ ]:
chaine1='bonjour'

chaine2='tout'
bin_repr = lambda s, coding="ascii": ' '.join('{0:08b}'.format(c) for c in s.encode(coding))
print(bin_repr(chaine1))
print(bin_repr(chaine2))

Alors j'ai trouvé ce code sur net mais il m'a donné la meme liste qu'avant mais ce n'est pas la meme chose que dans la définition précédante.

In [ ]:
def compte_lettre(mot):
    d = {}
    for c in mot:
        d[c] = d.get(c,0) + 1
    return d
def jaccard(mot1, mot2):
        d1 = compte_lettre(mot1)
        d2 = compte_lettre(mot2)

        suppression = {}
        for l in d1:
            c1 = d1[l]
            c2 = d2.get(l, 0) 
            if c2 != c1:
                suppression[l] = c2 - c1

        ajout = {}
        for l in d2:
            if l not in d1:
                c1 = 0
                c2 = d2[l]
                if c2 != c1:
                    ajout[l] = c2 - c1
            else:

                pass

        dist = sum(abs(x) for x in suppression.values()) + sum(abs(x) for x in ajout.values())
        return dist
    

In [ ]:
mot='alor'
if mot in listDesMots:
    print("il n'y a pas de faute d'orthographe yees")
else:
    mot1='$'+mot+'$'
    listeTriGramme=tri(mot1)
    listeDesMotCorr=set(listDesMots)
    for elem in listeTriGramme:
        if '$' not in elem:
            listeDesMotCorr=listeDesMotCorr&dicTrigramme[elem]
    listecorr=[]
    for elem in listeDesMotCorr:
        if jaccard(mot,elem)>0.2 :
            listecorr.append(elem)